In [2]:
import pandas as pd
import numpy as np
import json
import os

In [3]:
from utils import load_json, DotaTokenizer

project_root = os.path.dirname(os.getcwd())
hero_dpath = os.path.join(project_root, 'data', 'heroes.json')
match_dpath = os.path.join(project_root, 'data', 'main_metadata_2023.csv')
pickbans_dpath = os.path.join(project_root, 'data', 'picks_bans_2023.csv')

In [23]:
picks_df = pd.read_csv(pickbans_dpath)
matches_df = pd.read_csv(match_dpath)
hero_metadata = load_json(hero_dpath)

,is_pick,hero_id,team,order,match_id
48,False,59.0,1.0,0.0,6947166291
49,False,137.0,0.0,1.0,6947166291
50,False,136.0,1.0,2.0,6947166291
51,False,120.0,0.0,3.0,6947166291
52,True,11.0,1.0,4.0,6947166291
53,True,86.0,0.0,5.0,6947166291
54,True,28.0,0.0,6.0,6947166291
55,True,8.0,1.0,7.0,6947166291
56,True,70.0,0.0,8.0,6947166291
57,True,96.0,1.0,9.0,6947166291


In [28]:
pickarray = np.array(picks_df)

In [31]:
pickarray_unique_matches = np.unique(pickarray[:, 4])
pickarray_unique_matches

array([6947055162, 6947109699, 6947166291, ..., 7515557910, 7515575705,
       7515605198], dtype=object)

In [ ]:
def make_samples(pick_array: np.array, match_array: np.array, hero_metadata: dict, to_dataframe: bool):
    pass

In [5]:
special_tks = ['[PAD]', '[PICK]', '[BAN]', '[RADSTART]', '[DIRESTART]', '[RADWIN]', '[DIREWIN]']
tokenizer = DotaTokenizer(hero_metadata, special_tks)